In [4]:
%tensorflow_version 1.x
import pandas as pd
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import time
import numpy as np
import re
!pip install Sastrawi
# !pip install tqdm
from tqdm import tqdm 
import Sastrawi
import nltk 
from nltk.tokenize import word_tokenize, sent_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

TensorFlow 1.x selected.
     |████████████████████████████████| 215kB 8.2MB/s 


In [5]:
print(tf.__version__)

1.15.2


### Load & Preprocessing

In [6]:
#mount gdrive
path = '/content/drive/MyDrive/indosum.csv'

data = pd.read_csv(path)

In [7]:
data.head()

,paragraphs,summary
0,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...
2,"Jakarta , CNN Indonesia - - Dinas Pariwisata P...",Dinas Pariwisata Provinsi Bengkulu kembali men...
3,Merdeka.com - Indonesia Corruption Watch ( ICW...,Indonesia Corruption Watch ( ICW ) meminta Kom...
4,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,Jokowi memimpin upacara penurunan bendera .\nU...


In [8]:
data.shape

(14262, 2)

In [9]:
#display some of the data
with pd.option_context('display.max_colwidth', None):
  for i in range(3):
    print("Paragraph : " + data['paragraphs'].values[i])
    print("Summary : " + data['summary'].values[i] + "\n")
    

Paragraph : Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari .
Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu .Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia .
Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat ." Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit .
( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah .
Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) .Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh .
Dia juga tak tahu penyakit apa yang did

In [10]:
#data cleaning
def clean_data(txt):
  #1. Remove punctuations and special characters
  txt = re.sub(r'https?:\/\/.*[\r\n]*', '', txt, flags=re.MULTILINE)
  txt = re.sub(r'\<a href', ' ', txt)
  txt = re.sub(r'&amp;', '', txt) 
  txt = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', txt)
  txt = re.sub(r'<br />', ' ', txt)
  txt = re.sub(r'\'', ' ', txt)
  #2. Convert lower case
  txt = txt.lower()

  #3. Remove STOPWORDS (sastrawi(?) spacy id (?))
  txt = txt.split()
  txt = [word for word in txt if not word in stopwords]
  txt = ' '.join(txt)

  return txt

In [11]:
clean_summaries = []
for summary in tqdm(data.summary):
    clean_summaries.append(clean_data(summary))
print("Summaries are complete.")

clean_texts = []
for text in tqdm(data.paragraphs):
    clean_texts.append(clean_data(text))
print("Texts are complete.")

  2%|▏         | 214/14262 [00:00<00:06, 2131.49it/s]

Summaries are complete.


100%|██████████| 14262/14262 [00:06<00:00, 2132.26it/s]

Texts are complete.


In [12]:
for i in range(3):
  print("Paragraph : " + clean_summaries[i])
  print("Summary : " + clean_texts[i] + "\n")

Paragraph : dokter lula kamal merupakan selebriti sekaligus rekan kerja ryan thamrin menyebut kawannya sakit sejak setahun lalu lula menuturkan sakit membuat ryan mesti vakum semua kegiatannya termasuk menjadi pembawa acara dokter oz indonesia kondisi membuat ryan kampung halamannya pekanbaru riau menjalani istirahat
Summary : jakarta cnn indonesia dokter ryan thamrin terkenal lewat acara dokter oz indonesia meninggal dunia jumat 4 8 dini hari dokter lula kamal merupakan selebriti sekaligus rekan kerja ryan menyebut kawannya sakit sejak setahun lalu lula menuturkan sakit membuat ryan mesti vakum semua kegiatannya termasuk menjadi pembawa acara dokter oz indonesia kondisi membuat ryan kampung halamannya pekanbaru riau menjalani istirahat setahu orangnya sehat tahun lalu dengar sakit sakitnya langsung pulang pekanbaru jadi mau jenguk susah barangkali mau istirahat betul kalau jakarta susah isirahatnya kata lula cnnindonesia com jumat 4 8 lula mengenal ryan sejak aktif berkarier televisi 

In [13]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 99764


###Word Embedding

In [14]:
#using word2vec for indonesian trained with Wikipedia Indonesia datasets https://github.com/deryrahman/word2vec-bahasa-indonesia
# import gensim

# w2v_path = '/content/drive/MyDrive/idwiki_word2vec_100/idwiki_word2vec_100.model'
# model = gensim.models.word2vec.Word2Vec.load(w2v_path)
# model.wv.save_word2vec_format('test_w2v.txt', binary=False)


In [15]:
embeddings_index = {}
with open('/content/drive/MyDrive/idwiki_word2vec_100/idwiki_word2vec.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))


Word embeddings: 331793


In [16]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from word2vec:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from word2vec: 613
Percent of words that are missing from vocabulary: 0.61%


In [17]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 99764
Number of words we will use: 64666
Percent of words we will use: 64.82%


In [18]:
embedding_dim = 100 # Need to use 100 for embedding dimensions to match word2vec vectors.
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

64666


In [19]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [20]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 3844551
Total number of UNKs in headlines: 69971
Percent of words that are UNK: 1.82%


In [21]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [22]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
             counts
count  14262.000000
mean      46.488992
std        4.929286
min       26.000000
25%       43.000000
50%       46.000000
75%       50.000000
max       74.000000

Texts:
             counts
count  14262.000000
mean     224.077058
std       88.483104
min       26.000000
25%      164.000000
50%      209.000000
75%      267.000000
max      929.000000


In [23]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

336.0
387.0
516.0
53.0
55.0
58.38999999999942


In [24]:
def unk_counter(sentence):
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [25]:
sorted_summaries = []
sorted_texts = []
max_text_length = 334
max_summary_length = 58
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in tqdm(range(min(lengths_texts.counts), max_text_length)): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

100%|██████████| 308/308 [01:00<00:00,  5.07it/s]

3783
3783


### Building the Model

In [ ]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [ ]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
    #                                                                _zero_state_tensors(rnn_size, 
    #                                                                                    batch_size, 
    #                                                                                    tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [ ]:
#@title original seq2seqmodel

# def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
#                   vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    
    
#     embeddings = word_embedding_matrix
    
#     enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)

#     #encoding layer
#     for layer in range(num_layers):
#       with tf.compat.v1.variable_scope('encoder_{}'.format(layer)):
#         cell_fw = tf.keras.layers.LSTMCell(rnn_size)
#                                               #initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
#         cell_fw = tf.nn.RNNCellDropoutWrapper(cell_fw, 
#                                                     input_keep_prob = keep_prob)

#         cell_bw = tf.keras.layers.LSTMCell(rnn_size)
#                                               #initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
#         cell_bw = tf.nn.RNNCellDropoutWrapper(cell_bw, 
#                                                     input_keep_prob = keep_prob)

#         enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
#                                                                     cell_bw, 
#                                                                     enc_embed_input,
#                                                                     text_length,
#                                                                     dtype=tf.float32)
#     # Join outputs since we are using a bidirectional RNN
#     enc_output = tf.concat(enc_output,2)
#     # enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
#     #process encoding input
#     ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
#     dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

#     dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
#     #decoding layer
        
#     for layer in range(num_layers):
#         with tf.compat.v1.variable_scope('decoder_{}'.format(layer)):
#             lstm = tf.keras.layers.LSTMCell(rnn_size)
#                                            #initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
#             dec_cell = tf.nn.RNNCellDropoutWrapper(lstm, 
#                                                      input_keep_prob = keep_prob)
    
#     output_layer = Dense(vocab_size,
#                          kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
#     attn_mech = tfa.seq2seq.BahdanauAttention(rnn_size,
#                                                   enc_output,
#                                                   text_length,
#                                                   normalize=False,
#                                                   name='BahdanauAttention')

#     dec_cell = tfa.seq2seq.AttentionWrapper(dec_cell,
#                                                           attn_mech,
#                                                           rnn_size)
            
#     #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
#     #                                                                _zero_state_tensors(rnn_size, 
#     #                                                                                    batch_size, 
#     #                                                                                    tf.float32)) 
#     initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

#     with tf.compat.v1.variable_scope("decode"):
#         train_sampler = tfa.seq2seq.TrainingSampler(time_major=False)
#         training_helper = sampler.initialize(inputs=dec_embed_input,
#                                                         text_length=summary_length) 
      
#         training_decoder = tfa.seq2seq.BasicDecoder(dec_cell,
#                                                        training_helper,
#                                                        initial_state,
#                                                        output_layer) 
      
#         training_logits, _ , _ = tfa.seq2seq.dynamic_decode(training_decoder,
#                                                            output_time_major=False,
#                                                            impute_finished=True,
#                                                            maximum_iterations=max_summary_length)
        
#         # training_decoder = training_decoding_layer(dec_embed_input, 
#         #                                           summary_length, 
#         #                                           dec_cell, 
#         #                                           initial_state,
#         #                                           output_layer,
#         #                                           vocab_size, 
#         #                                           max_summary_length)
        
#         training_logits,_ ,_ = tfa.seq2seq.dynamic_decode(training_decoder,
#                                   output_time_major=False,
#                                   impute_finished=True,
#                                   maximum_iterations=max_summary_length)
        
#     with tf.compat.v1.variable_scope("decode", reuse=True):
            
#         start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
#         inf_sampler = tfa.seq2seq.GreedyEmbeddingSampler(embeddings)
#         inference_helper = inf_sampler.initialize(start_tokens, end_token)
                
#         inference_decoder = tfa.seq2seq.BasicDecoder(dec_cell,
#                                                         inference_helper,
#                                                         initial_state,
#                                                         output_layer)
                
#         inference_logits, _ , _ = tfa.seq2seq.dynamic_decode(inference_decoder,
#                                                             output_time_major=False,
#                                                             impute_finished=True,
#                                                             maximum_iterations=max_summary_length)
#         # inference_decoder = inference_decoding_layer(embeddings,  
#         #                                             vocab_to_int['<GO>'], 
#         #                                             vocab_to_int['<EOS>'],
#         #                                             dec_cell, 
#         #                                             initial_state, 
#         #                                             output_layer,
#         #                                             max_summary_length,
#         #                                             batch_size)
        
#         inference_logits,_ ,_ = tfa.seq2seq.dynamic_decode(inference_decoder,
#                                   output_time_major=False,
#                                   impute_finished=True,
#                                   maximum_iterations=max_summary_length)
#     # training_logits, inference_logits  = decoding_layer(dec_embed_input, 
#     #                                                     embeddings,
#     #                                                     enc_output,
#     #                                                     enc_state, 
#     #                                                     vocab_size, 
#     #                                                     text_length, 
#     #                                                     summary_length, 
#     #                                                     max_summary_length,
#     #                                                     rnn_size, 
#     #                                                     vocab_to_int, 
#     #                                                     keep_prob, 
#     #                                                     batch_size,
#     #                                                     num_layers)
    
#     return training_logits, inference_logits

In [ ]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

###Hyperparameters

In [ ]:
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 3
learning_rate = 0.008
keep_probability = 0.75

In [ ]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Gr

###Training

In [ ]:
print("The shortest text length:", len(sorted_texts[0]))
print("The longest text length:",len(sorted_texts[-1]))

The shortest text length: 41
The longest text length: 333


In [ ]:
learning_rate_decay = 0.95
min_learning_rate = 5e-4
display_step = 20
stop_early = 0
stop = 6 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3
update_check = (len(sorted_texts)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = []

In [ ]:
tf.reset_default_graph()
checkpoint = "/content/drive/MyDrive/Checkpoints"  #100k sentence"  #300k sentence
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    #sess.run(tf.local_variables_initializer())

    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries, sorted_texts, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0
                
                #saver = tf.train.Saver() 
                #saver.save(sess, checkpoint)
                
            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
              
                  
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Average loss for this update: 8.788
New Record!
Epoch   1/100 Batch   20/59 - Loss:  8.631, Seconds: 13.26
Average loss for this update: 6.997
New Record!
Epoch   1/100 Batch   40/59 - Loss:  6.981, Seconds: 15.22
Average loss for this update: 6.781
New Record!
Average loss for this update: 6.441
New Record!
Epoch   2/100 Batch   20/59 - Loss:  6.388, Seconds: 13.64
Average loss for this update: 5.751
New Record!
Epoch   2/100 Batch   40/59 - Loss:  5.746, Seconds: 15.53
Average loss for this update: 5.64
New Record!
Average loss for this update: 5.606
New Record!
Epoch   3/100 Batch   20/59 - Loss:  5.562, Seconds: 13.79
Average loss for this update: 5.046
New Record!
Epoch   3/100 Batch   40/59 - Loss:  5.045, Seconds: 15.48
Average loss for this update: 4.991
New Record!
Average loss for this update: 5.02
No Improvement.
Epoch   4/100 Batch   20/59 - Loss:  4.987, Seconds: 13.70
Average loss for this update: 4.574
New Record!
Epoch   4/100 Batch   40/59 - Loss:  4.562, Seconds: 15.3

In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
checkpoint = "/content/drive/MyDrive/Checkpoints/Checkpoints" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in loaded_graph.as_graph_def().node]
names

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Checkpoints/Checkpoints


['input',
 'targets',
 'learning_rate',
 'keep_prob',
 'summary_length',
 'Const',
 'max_dec_len',
 'text_length',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'encoder_0/DropoutWrapperInit/Const',
 'encoder_0/DropoutWrapperInit/Const_1',
 'encoder_0/DropoutWrapperInit_1/Const',
 'encoder_0/DropoutWrapperInit_1/Const_1',
 'encoder_0/bidirectional_rnn/fw/fw/Rank',
 'encoder_0/bidirectional_rnn/fw/fw/range/start',
 'encoder_0/bidirectional_rnn/fw/fw/range/delta',
 'encoder_0/bidirectional_rnn/fw/fw/range',
 'encoder_0/bidirectional_rnn/fw/fw/concat/values_0',
 'encoder_0/bidirectional_rnn/fw/fw/concat/axis',
 'encoder_0/bidirectional_rnn/fw/fw/concat',
 'encoder_0/bidirectional_rnn/fw/fw/transpose',
 'encoder_0/bidirectional_rnn/fw/fw/sequence_length',
 'encoder_0/bidirectional_rnn/fw/fw/Shape',
 'encoder_0/bidirectional_rnn/fw/fw/strided_slice/stack',
 'encoder_0/bidirectional_rnn/fw/fw/strid

###Test

In [2]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_data(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [79]:
# Create your own review or use one from the dataset
input_sentence = """UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA TAHUN 1945
PEMBUKAAN
( P r e a m b u l e)


Bahwa sesungguhnya Kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu, maka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan perikeadilan.


Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat yang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan pintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil dan makmur.


Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan luhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan dengan ini kemerdekaannya.


Kemudian daripada itu untuk membentuk suatu Pemerintah Negara Indonesia yang melindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk memajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut melaksanakan ketertiban dunia yang berdasarkan kemerdekaan, perdamaian abadi dan keadilan sosial, maka disusunlah Kemerdekaan Kebangsaan Indonesia itu dalam suatu Undang-Undang Dasar Negara Indonesia, yang terbentuk dalam suatu susunan Negara Republik Indonesia yang berkedaulatan rakyat dengan berdasar kepada Ketuhanan Yang Maha Esa, Kemanusiaan yang adil dan beradab, Persatuan Indonesia dan Kerakyatan yang dipimpin oleh hikmat kebijaksanaan dalam Permusyawaratan/Perwakilan, serta dengan mewujudkan suatu Keadilan sosial bagi seluruh rakyat Indonesia."""
text = text_to_seq(input_sentence)
# random = np.random.randint(0,len(clean_texts))
# input_sentence = clean_texts[random]
# text = text_to_seq(clean_texts[random])

checkpoint = "/content/drive/MyDrive/Checkpoints/summarizer" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    summary_len = round(len(text)/2)
    if(summary_len > 55):
      summary_len = 55

    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [summary_len], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

# print('Original Text:', reviews.Text[random])
# print('Original summary:', reviews.Summary[random])#clean_summaries[random]

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Checkpoints/summarizer

Text
  Word Ids:    [1883, 1883, 2121, 160, 3057, 27, 200, 2157, 2734, 10752, 3732, 110, 859, 2439, 1480, 1305, 8591, 110, 14413, 4428, 1202, 786, 2668, 184, 31816, 56, 445, 17235, 917, 36519, 64662, 4679, 1970, 4428, 27, 45718, 5726, 2528, 64662, 627, 4101, 27, 46, 589, 3125, 4428, 160, 27, 4429, 1451, 26718, 5991, 23118, 56, 3644, 33059, 6500, 2127, 510, 64662, 1758, 32191, 64662, 946, 1375, 4101, 27, 1113, 23361, 367, 635, 4055, 91, 160, 27, 3248, 13183, 184, 27, 677, 7797, 1687, 27, 10675, 2433, 1111, 31458, 2553, 184, 102, 3003, 12160, 445, 737, 4428, 4973, 7197, 2123, 1037, 64662, 4428, 946, 27, 4055, 1883, 1883, 2121, 160, 27, 11868, 4055, 15616, 160, 3057, 27, 64662, 4101, 12047, 16694, 2127, 2128, 9485, 5991, 34444, 1458, 27, 32748, 794, 23226, 37393, 6275, 1871, 7098, 4055, 2123, 1037, 677, 4101, 27]
  Input Words: undang undang dasar negara republik indonesia tahun 1945 pembukaan p r e a

In [31]:
for i in range(3):
  print(vocab_to_int["undang"])

1883
1883
1883


64
